In [1]:
import xlwings as xw
import pandas as pd

# Conecta ao Excel já aberto
wb = xw.books['8.0_Relatorio_fechamento junho_2025_VERSÃO FINAL 24-07-2025.xlsm']

In [7]:
range_nomeado = 'risco_cart_1'
rng = wb.names[range_nomeado].refers_to_range

data = rng.options(pd.DataFrame, header=3, index=False).value

if isinstance(data.columns, pd.MultiIndex):
    data.columns = [col[-1] for col in data.columns]

print('Colunas:', data.columns.tolist())

# Função para buscar coluna que contenha uma string, ignorando acento e case
def achar_coluna_nome(df, texto):
    import unicodedata
    def normaliza(s):  # Remove acentos e põe minúsculo
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        ).lower()
    alvo = normaliza(texto)
    for c in df.columns:
        if alvo in normaliza(str(c)):
            return c
    raise Exception(f'Coluna contendo "{texto}" não encontrada.')

# Busca nomes corretos:
col_data      = achar_coluna_nome(data, 'data')
col_patr      = achar_coluna_nome(data, 'patrimônio')
col_pnl_perc  = achar_coluna_nome(data, 'Dia')  # ou ajuste o texto
try:
    col_pnl_fin   = achar_coluna_nome(data, 'PnL Dia')   # ou ajuste o texto
except:
    col_pnl_fin   = achar_coluna_nome(data, '1D')   # ou ajuste o texto

data[col_data] = pd.to_datetime(data[col_data])

inicio = '2024-12-31'
fim = '2025-06-30'
filtro = (data[col_data] >= inicio) & (data[col_data] <= fim)
df_out = data.loc[filtro, [col_data, col_patr, col_pnl_perc, col_pnl_fin]]

display(df_out)
df_out.to_clipboard()

Colunas: ['Data', 'Entradas', 'Saídas', 'Patrimonio', 'Dia', 'Mês', 'Ano', 'PnL Dia', 'PnL Mês', 'PnL Ano']


,Data,Patrimonio,Dia,PnL Dia
253,2024-12-31,5.481264e+09,0.000384,2.102174e+06
254,2025-01-02,5.481599e+09,0.000061,3.347690e+05
255,2025-01-03,5.492767e+09,0.002037,1.116838e+07
256,2025-01-06,5.511269e+09,0.003368,1.850237e+07
257,2025-01-07,5.456889e+09,-0.000795,-4.380134e+06
...,...,...,...,...
371,2025-06-24,5.693601e+09,0.002359,1.339680e+07
372,2025-06-25,5.668767e+09,-0.004362,-2.483329e+07
373,2025-06-26,5.682136e+09,0.002358,1.336912e+07
374,2025-06-27,5.675354e+09,-0.001194,-6.782848e+06


In [5]:
df_out.to_clipboard()